# Unit 8a — Advanced OOP Techniques

**Purpose:** Build on OOP foundations with professional techniques for writing **robust**, **maintainable**, and **scalable** code—practices widely used in industry software development.

---

## 📚 Table of Contents

1. [Type Hints — Writing Self-Documenting Code](#1.-Type-Hints-—-Writing-Self-Documenting-Code)
2. [Encapsulation & Access Control](#2.-Encapsulation-&-Access-Control)
3. [Private Attributes & Name Mangling](#3.-Private-Attributes-&-Name-Mangling)
4. [Properties — Pythonic Attribute Access](#4.-Properties-—-Pythonic-Attribute-Access)
5. [Dataclasses — Reducing Boilerplate](#5.-Dataclasses-—-Reducing-Boilerplate)
6. [Abstract Base Classes (ABC)](#6.-Abstract-Base-Classes-(ABC))
7. [Class Methods and Static Methods](#7.-Class-Methods-and-Static-Methods)
8. [Exercises](#8.-Exercises)

---

## Prerequisites

Before starting this unit, you should be comfortable with:
- Unit 7 (OOP Foundations) — classes, objects, inheritance basics

## 🎯 Learning Objectives

By the end of this unit, you will be able to:

**📝 Type Hints**
- Annotate functions and classes with type hints for better readability
- Use modern union syntax (`X | None`, `A | B`) for optional and union types
- Understand type hints as documentation for developers and tools

**🔒 Encapsulation & Data Protection**
- Apply encapsulation to protect object invariants
- Use properties to validate and control attribute access
- Choose appropriate access conventions (`_attr`, `__attr`)

**📦 Dataclasses**
- Use `@dataclass` to reduce boilerplate in data-holding classes
- Configure dataclasses with `field()`, `frozen=True`, and `__post_init__`
- Combine dataclasses with properties for validation

**📋 Interfaces & Contracts**
- Define interfaces and contracts using Abstract Base Classes
- Program to interfaces for flexibility and testability

**🔧 Method Types**
- Use `@classmethod` for alternative constructors and class-level operations
- Use `@staticmethod` for utility functions grouped with a class

---

# 1. Type Hints — Writing Self-Documenting Code

## 1.1 Why Type Hints Matter

Python is dynamically typed, but **type hints** (introduced in Python 3.5+) allow you to annotate expected types without enforcing them at runtime. This provides:

| Benefit | Description |
|---------|-------------|
| **Readability** | Code documents itself — readers know what types are expected |
| **IDE Support** | Better autocomplete, error detection, and refactoring |
| **Bug Prevention** | Static analyzers (mypy, Pylance) catch type mismatches early |
| **Team Collaboration** | Clear contracts between functions and modules |

> **Note:** Type hints are *optional* and *not enforced* by Python at runtime. They are metadata for developers and tools.

In [ ]:
# Basic type hints
def greet(name: str) -> str:
    """Greet a person by name."""
    return f"Hello, {name}!"

# Without type hints - unclear what types are expected
def calculate_area_bad(width, height):
    return width * height

# With type hints - self-documenting
def calculate_area(width: float, height: float) -> float:
    """Calculate rectangle area."""
    return width * height

# Type hints for variables (less common, but useful)
count: int = 0
ratio: float = 0.5
name: str = "Alice"
is_active: bool = True

# Examples
print(greet("Alice"))
print(f"Area: {calculate_area(4.5, 3.0)}")

## 1.2 Common Type Annotations

Python 3.10+ introduced modern union syntax using `|` (pipe operator). This is the recommended approach:

In [ ]:
# Collections - use built-in types (Python 3.9+)
def process_items(items: list[str]) -> dict[str, int]:
    """Count occurrences of each item."""
    return {item: items.count(item) for item in set(items)}

# Optional - value can be None (Python 3.10+ syntax)
def find_user(user_id: int) -> str | None:
    """Find user by ID, returns None if not found."""
    users = {1: "Alice", 2: "Bob"}
    return users.get(user_id)  # Returns None if not found

# Union - multiple possible types (Python 3.10+ syntax)
def format_value(value: int | float | str) -> str:
    """Format various types as string."""
    if isinstance(value, (int, float)):
        return f"{value:,.2f}"
    return str(value)

# Examples
print(process_items(["a", "b", "a", "c", "a"]))
print(f"User 1: {find_user(1)}")
print(f"User 99: {find_user(99)}")
print(f"Formatted: {format_value(1234567.8)}")

## 1.3 Type Hints in Classes

Type hints are especially valuable in object-oriented code, making class interfaces clear:

In [ ]:
class ShoppingCart:
    """A shopping cart with typed methods."""
    
    def __init__(self, customer_name: str) -> None:
        self.customer_name: str = customer_name
        self.items: list[tuple[str, float, int]] = []  # (name, price, quantity)
    
    def add_item(self, name: str, price: float, quantity: int = 1) -> None:
        """Add an item to the cart."""
        self.items.append((name, price, quantity))
    
    def get_total(self) -> float:
        """Calculate total price."""
        return sum(price * qty for _, price, qty in self.items)
    
    def get_item_count(self) -> int:
        """Get total number of items."""
        return sum(qty for _, _, qty in self.items)
    
    def find_item(self, name: str) -> tuple[str, float, int] | None:
        """Find item by name, returns None if not found."""
        for item in self.items:
            if item[0] == name:
                return item
        return None


# Usage - IDE now provides excellent autocomplete and error checking
cart = ShoppingCart("Alice")
cart.add_item("Laptop", 999.99)
cart.add_item("Mouse", 29.99, 2)

print(f"Customer: {cart.customer_name}")
print(f"Total items: {cart.get_item_count()}")
print(f"Total price: ${cart.get_total():.2f}")
print(f"Find 'Mouse': {cart.find_item('Mouse')}")

---

# 2. Encapsulation & Access Control

## 2.1 What is Encapsulation?

**Encapsulation** is one of the four pillars of OOP (along with Abstraction, Inheritance, and Polymorphism).

> **Definition:** Encapsulation means bundling data (attributes) and methods that operate on that data within a single unit (class), while restricting direct access to some components.

### Benefits of Encapsulation

| Problem Without Encapsulation | Solution With Encapsulation |
|------------------------------|----------------------------|
| Anyone can set invalid values | Validation in setters/methods |
| Internal changes break external code | Stable public interface |
| Hard to track where data changes | All changes go through methods |
| Difficult to add logging/validation later | Central point of control |

### Python's Approach to Access Control

Python does **not** enforce access modifiers like Java/C# (`private`, `protected`, `public`).
Instead, it uses **naming conventions**:

| Convention | Meaning | Example |
|------------|---------|--------|
| `attr` | Public — part of the API | `self.name` |
| `_attr` | Protected/Internal — use within class or subclasses | `self._balance` |
| `__attr` | Private — name-mangled to prevent accidental access | `self.__secret` |

> **Note:** Python follows the principle "We're all consenting adults here." The language trusts developers to follow conventions rather than enforcing strict access control.

## 2.2 Example: Bank Account Without Protection

In [ ]:
# Without encapsulation - anyone can modify balance directly
class BankAccount:
    def __init__(self, owner, balance=0.0):
        self.owner = owner
        self.balance = balance  # Public attribute - no protection!

# Creating an account
acc = BankAccount("Alice", 100)
print(f"Owner: {acc.owner}")
print(f"Balance: ${acc.balance}")

# Problem: Nothing prevents invalid operations!
acc.balance = -999  # This should not be allowed!
print(f"After invalid change: ${acc.balance}")  # Oops!

## 2.3 Protected Attributes (Single Underscore)

The single underscore `_attr` signals that an attribute is intended for internal use. While not enforced by the interpreter, professional Python developers respect this convention.

In [ ]:
# With encapsulation - balance is protected and accessed via methods
class SafeAccount:
    """A bank account with protected balance and validation."""
    
    def __init__(self, owner: str, balance: float = 0.0):
        self.owner = owner
        self._balance = float(balance)  # Protected: underscore convention

    def deposit(self, amount: float) -> float:
        """Add money to the account."""
        amount = float(amount)
        if amount <= 0:
            raise ValueError("Deposit amount must be positive")
        self._balance += amount
        return self._balance

    def withdraw(self, amount: float) -> float:
        """Remove money from the account."""
        amount = float(amount)
        if amount <= 0:
            raise ValueError("Withdrawal amount must be positive")
        if amount > self._balance:
            raise ValueError(f"Insufficient funds. Available: ${self._balance}")
        self._balance -= amount
        return self._balance

    def get_balance(self) -> float:
        """Return current balance (read-only access)."""
        return self._balance

# Usage
account = SafeAccount("Bob", 100)
print(f"Initial balance: ${account.get_balance()}")

account.deposit(50)
print(f"After deposit: ${account.get_balance()}")

account.withdraw(30)
print(f"After withdrawal: ${account.get_balance()}")

# Try invalid operations
try:
    account.deposit(-50)
except ValueError as e:
    print(f"Error: {e}")

try:
    account.withdraw(500)
except ValueError as e:
    print(f"Error: {e}")

---

# 3. Private Attributes & Name Mangling

## 3.1 Double Underscore Prefix: Name Mangling

When you use `__attr` (double underscore prefix), Python performs **name mangling**:
- `self.__balance` becomes `self._ClassName__balance`

This mechanism prevents **accidental** access and avoids name collisions in inheritance hierarchies.

> **Important:** This is not true privacy—it is designed to prevent accidents, not to secure data from determined access.

In [ ]:
# Demonstrating name mangling
class SecureAccount:
    def __init__(self, owner: str, balance: float = 0.0):
        self.owner = owner
        self.__balance = float(balance)  # Name-mangled attribute

    def deposit(self, amount: float) -> None:
        if amount <= 0:
            raise ValueError("Amount must be positive")
        self.__balance += float(amount)

    def get_balance(self) -> float:
        return self.__balance

# Create account
acc = SecureAccount("Alice", 100)
acc.deposit(50)
print(f"Balance via method: ${acc.get_balance()}")

# Direct access fails
try:
    print(acc.__balance)  # AttributeError!
except AttributeError as e:
    print(f"Direct access blocked: {e}")

# But name-mangled access still works (NOT recommended!)
print(f"Mangled name access: ${acc._SecureAccount__balance}")

# Show all attributes
print(f"\nAll attributes: {[attr for attr in dir(acc) if 'balance' in attr.lower()]}")

## 3.2 Name Mangling in Inheritance

Name mangling prevents attribute name collisions between parent and child classes:

In [ ]:
# Name mangling prevents accidental overrides in inheritance
class Parent:
    def __init__(self):
        self.__secret = "Parent's secret"
    
    def reveal_parent_secret(self) -> str:
        return self.__secret

class Child(Parent):
    def __init__(self):
        super().__init__()
        self.__secret = "Child's secret"  # Different attribute!
    
    def reveal_child_secret(self) -> str:
        return self.__secret

child = Child()
print(f"Parent's secret: {child.reveal_parent_secret()}")
print(f"Child's secret: {child.reveal_child_secret()}")

# Both secrets exist independently due to name mangling
print(f"\nAttributes: {[a for a in dir(child) if 'secret' in a]}")

## 3.3 Choosing the Right Convention

| Convention | Use Case | Example |
|------------|----------|--------|
| `attr` | Part of public API | `self.name`, `self.owner` |
| `_attr` | Internal implementation detail | `self._cache`, `self._balance` |
| `__attr` | Avoiding name clashes in inheritance | `self.__id` in base class |

> **Best Practice:** Start with `_attr` for internal attributes. Only use `__attr` when you specifically need to prevent inheritance conflicts.

---

# 4. Properties — Pythonic Attribute Access

## 4.1 The Problem with Getters and Setters

In languages like Java and C#, explicit getter/setter methods are common:
```java
public double getBalance() { return this.balance; }
public void setBalance(double value) { this.balance = value; }
```

This approach is verbose. Python provides a more elegant solution: **properties**.

## 4.2 What Are Properties?

Properties allow you to:
- Access attributes with **simple syntax** (`obj.attr`)
- Execute **validation logic** behind the scenes
- Create **read-only** attributes
- Compute **derived values** on-the-fly

Properties are defined using the `@property` decorator.

In [ ]:
# Temperature class with property validation
class Temperature:
    """Temperature with automatic validation and unit conversion."""
    
    ABSOLUTE_ZERO = -273.15  # Kelvin = 0
    
    def __init__(self, celsius: float):
        self._celsius = celsius  # Uses the setter!

    @property
    def celsius(self) -> float:
        """Get temperature in Celsius."""
        return self._celsius

    @celsius.setter
    def celsius(self, value: float) -> None:
        """Set temperature in Celsius with validation."""
        value = float(value)
        if value < self.ABSOLUTE_ZERO:
            raise ValueError(f"Temperature cannot be below absolute zero ({self.ABSOLUTE_ZERO}°C)")
        self._celsius = value

    @property
    def fahrenheit(self) -> float:
        """Get temperature in Fahrenheit (computed property)."""
        return self._celsius * 9/5 + 32
    
    @fahrenheit.setter
    def fahrenheit(self, value: float) -> None:
        """Set temperature using Fahrenheit."""
        self._celsius = (float(value) - 32) * 5/9  # Converts and validates
    
    @property
    def kelvin(self) -> float:
        """Get temperature in Kelvin."""
        return self._celsius - self.ABSOLUTE_ZERO
    
    def __repr__(self) -> str:
        return f"Temperature({self._celsius}°C / {self.fahrenheit}°F / {self.kelvin}K)"

# Usage examples
t = Temperature(20)
print(f"Created: {t}")

print(f"Celsius: {t.celsius}°C")
print(f"Fahrenheit: {t.fahrenheit}°F")
print(f"Kelvin: {t.kelvin}K")


t.celsius = 25
print(f"After setting Celsius: {t}")

t.fahrenheit = 98.6  # Human body temperature
print(f"After setting Fahrenheit: {t}")

# Try invalid value
try:
    t.celsius = -300  # Below absolute zero
except ValueError as e:
    print(f"Validation error: {e}")

## 4.3 Read-Only Properties

Defining only a getter (no setter) makes an attribute **read-only**:

In [ ]:
# Read-only properties example
class User:
    """User with immutable username and computed email."""
    
    def __init__(self, username: str, domain: str = "company.com"):
        self._username = username
        self._domain = domain
        self._created_at = "2024-01-15"  # Simulated timestamp

    @property
    def username(self) -> str:
        """Username is read-only after creation."""
        return self._username
    
    @property
    def email(self) -> str:
        """Email is computed from username and domain."""
        return f"{self._username.lower()}@{self._domain}"
    
    @property
    def created_at(self) -> str:
        """Creation timestamp is immutable."""
        return self._created_at

# Usage
user = User("AliceSmith")
print(f"Username: {user.username}")
print(f"Email: {user.email}")
print(f"Created: {user.created_at}")

# Try to modify read-only property
try:
    user.username = "BobJones"
except AttributeError as e:
    print(f"\nCannot modify: {e}")

## 4.4 Caching with Properties (Lazy Loading)

Properties can implement **lazy loading** — computing expensive values only when first accessed, then caching the result for future use. When the underlying data changes, the cache is automatically invalidated.

In [ ]:
class CachedData:
    """Demonstrates property with cache clearing via deleter."""
    
    def __init__(self, raw_data: list[int]):
        self._raw_data = raw_data
        self._squaredList: list[int] | None = None  # Cache
    
    @property
    def raw_data(self) -> list[int]:
        return self._raw_data

    @raw_data.setter
    def raw_data(self, newList: list[int]) -> None:
        self._squaredList = None
        self._raw_data = newList
    
    @property
    def squaredList(self) -> list[int]:
        """Lazy-computed processed data with caching."""
        if self._squaredList is None:
            print("Computing processed data...")
            self._squaredList = [x ** 2 for x in self._raw_data]  # Expensive operation
        return self._squaredList.copy()


# Usage
data = CachedData([1, 2, 3, 4, 5, 0, -6, 7, 8, 9, 10])
print("First Call")
%time data.squaredList  # Computes

print("\nSecond Call")
%time data.squaredList  # Uses cache

print(f"\nSquared List: {data.squaredList}")

## 4.5 Property with Deleter

You can also define behavior for `del obj.attr`:

In [ ]:
# Complete property example with getter, setter, and deleter
class CachedDataWithDeleter:
    """Demonstrates property with cache clearing via deleter."""
    
    def __init__(self, raw_data: list[int]):
        self._raw_data = raw_data
        self._processed: list[int] | None = None  # Cache
    
    @property
    def processed(self) -> list[int]:
        """Lazy-computed processed data with caching."""
        if self._processed is None:
            print("Computing processed data...")
            self._processed = [x * 2 for x in self._raw_data]  # Expensive operation
        return self._processed
    
    @processed.deleter
    def processed(self) -> None:
        """Clear the cache."""
        print("Clearing cache...")
        self._processed = None

# Usage
data = CachedDataWithDeleter([1, 2, 3, 4, 5])
%time data.processed  # Computes
%time data.processed  # Uses cache

print(f"\nProcessed Data: {data.processed}\n")

del data.processed  # Clear cache
%time print(data.processed)  # Recomputes

---

# 5. Dataclasses — Reducing Boilerplate

## 5.1 The Problem with Traditional Classes

When creating simple data-holding classes, you often write repetitive boilerplate:

In [ ]:
# Traditional class - lots of boilerplate!
class ProductTraditional:
    """A product class written the traditional way."""
    
    def __init__(self, name: str, price: float, quantity: int = 0):
        self.name = name
        self.price = price
        self.quantity = quantity
    
    def __repr__(self) -> str:
        return f"ProductTraditional(name={self.name!r}, price={self.price}, quantity={self.quantity})"
    
    def __eq__(self, other: object) -> bool:
        if not isinstance(other, ProductTraditional):
            return NotImplemented
        return self.name == other.name and self.price == other.price and self.quantity == other.quantity


# Creating and comparing products
p1 = ProductTraditional("Laptop", 999.99, 5)
p2 = ProductTraditional("Laptop", 999.99, 5)

print(f"Product: {p1}")
print(f"Equal? {p1 == p2}")  # Works because we defined __eq__

## 5.2 Introducing Dataclasses

The `@dataclass` decorator (Python 3.7+) automatically generates `__init__`, `__repr__`, `__eq__`, and more:

In [ ]:
from dataclasses import dataclass, field

@dataclass
class Product:
    """A product - same functionality with minimal code!"""
    name: str
    price: float
    quantity: int = 0  # Default value


# Same behavior, much less code!
p1 = Product("Laptop", 999.99, 5)
p2 = Product("Laptop", 999.99, 5)
p3 = Product("Mouse", 29.99)

print(f"Product: {p1}")           # Auto-generated __repr__
print(f"Equal? {p1 == p2}")       # Auto-generated __eq__
print(f"Mouse: {p3}")             # Default quantity = 0

## 5.3 Dataclass Options and Features

Dataclasses offer many customization options through decorator parameters and the `field()` function:

In [ ]:
from dataclasses import dataclass, field
from datetime import datetime

@dataclass
class Order:
    """An order with various dataclass features."""
    order_id: int
    customer: str
    items: list[str] = field(default_factory=list)  # Mutable default - use field()!
    total: float = 0.0
    notes: str | None = None
    created_at: datetime = field(default_factory=datetime.now)
    
    # Computed field (excluded from __init__)
    _processed: bool = field(default=False, repr=False)
    
    def add_item(self, item: str, price: float) -> None:
        """Add item and update total."""
        self.items.append(item)
        self.total += price
    
    def mark_processed(self) -> None:
        """Mark order as processed."""
        self._processed = True


# Create and modify orders
order1 = Order(1001, "Alice")
order1.add_item("Laptop", 999.99)
order1.add_item("Mouse", 29.99)

order2 = Order(1002, "Bob", notes="Express delivery")

print(f"Order 1: {order1}")
print(f"Order 2: {order2}")
print(f"Order 1 items: {order1.items}")
print(f"Order 2 items: {order2.items}")  # Empty - not shared!

## 5.4 Frozen Dataclasses (Immutable)

Use `frozen=True` to create immutable instances — attempting to modify attributes raises an error:

In [ ]:
@dataclass(frozen=True)
class Point:
    """An immutable 2D point - can be used as dict key or in sets."""
    x: float
    y: float
    
    def distance_to(self, other: "Point") -> float:
        """Calculate distance to another point."""
        return ((self.x - other.x) ** 2 + (self.y - other.y) ** 2) ** 0.5


# Frozen dataclasses are immutable
p1 = Point(3.0, 4.0)
p2 = Point(0.0, 0.0)

print(f"Point: {p1}")
print(f"Distance from origin: {p1.distance_to(p2)}")

# Can be used as dict keys (hashable)!
distances = {p1: "far", p2: "origin"}
print(f"Point lookup: {distances[p1]}")

# Cannot modify frozen dataclass
try:
    p1.x = 10.0
except AttributeError as e:
    print(f"Cannot modify frozen: {e}")

## 5.5 Dataclasses vs Properties

Dataclasses work well with properties for validation. You can add a `__post_init__` method to perform validation after automatic initialization:

In [ ]:
@dataclass
class ValidatedProduct:
    """Product with validation using __post_init__."""
    name: str
    price: float
    quantity: int = 0
    
    def __post_init__(self) -> None:
        """Validate fields after __init__ runs."""
        if not self.name:
            raise ValueError("Name cannot be empty")
        if self.price <= 0:
            raise ValueError("Price must be positive")
        if self.quantity < 0:
            raise ValueError("Quantity cannot be negative")
    
    @property
    def total_value(self) -> float:
        """Computed property works normally with dataclasses."""
        return self.price * self.quantity


# Valid product
product = ValidatedProduct("Laptop", 999.99, 5)
print(f"Product: {product}")
print(f"Total value: ${product.total_value:.2f}")

# Invalid product raises error
try:
    bad_product = ValidatedProduct("Laptop", -100)
except ValueError as e:
    print(f"Validation error: {e}")

---

# 6. Abstract Base Classes (ABC)

## 6.1 Enforcing Interfaces

Consider building a system where different storage backends must implement `save()` and `load()`. How do you ensure all implementations follow this contract?

**Without ABCs:**
- No compile-time or import-time checks
- Errors only appear at runtime when methods are called
- Documentation is the only "contract"

**With ABCs:**
- Python raises `TypeError` if you try to instantiate a class without implementing all abstract methods
- Clear contract visible in code
- IDE support for detecting missing methods

## 6.2 Creating an Abstract Base Class

In [ ]:
from abc import ABC, abstractmethod

class Storage(ABC):
    """Abstract base class defining storage interface.
    
    Any class inheriting from Storage MUST implement save() and load().
    """
    
    @abstractmethod
    def save(self, data) -> None:
        """Save data to storage."""
        pass

    @abstractmethod
    def load(self):
        """Load and return data from storage."""
        pass
    
    # Non-abstract method - provides default implementation
    def exists(self) -> bool:
        """Check if storage has data (can be overridden)."""
        try:
            return self.load() is not None
        except:
            return False

# Try to instantiate abstract class directly
try:
    s = Storage()
except TypeError as e:
    print(f"Cannot instantiate abstract class: {e}")

## 6.3 Implementing the Interface

The following concrete implementations satisfy the `Storage` contract:

In [ ]:
import json
import csv
from pathlib import Path

class JSONStorage(Storage):
    """Store data as JSON file."""
    
    def __init__(self, filepath: str):
        self.path = Path(filepath)

    def save(self, data) -> None:
        self.path.parent.mkdir(parents=True, exist_ok=True)
        with self.path.open("w", encoding="utf-8") as f:
            json.dump(data, f, ensure_ascii=False, indent=2)
        print(f"Saved to {self.path}")

    def load(self):
        if not self.path.exists():
            return []
        with self.path.open("r", encoding="utf-8") as f:
            return json.load(f)


class CSVStorage(Storage):
    """Store data as CSV file."""
    
    def __init__(self, filepath: str, fieldnames: list[str]):
        self.path = Path(filepath)
        self.fieldnames = list(fieldnames)

    def save(self, data) -> None:
        self.path.parent.mkdir(parents=True, exist_ok=True)
        with self.path.open("w", newline="", encoding="utf-8") as f:
            writer = csv.DictWriter(f, fieldnames=self.fieldnames)
            writer.writeheader()
            writer.writerows(data)
        print(f"Saved to {self.path}")

    def load(self):
        if not self.path.exists():
            return []
        with self.path.open("r", newline="", encoding="utf-8") as f:
            return list(csv.DictReader(f))


class MemoryStorage(Storage):
    """Store data in memory (useful for testing)."""
    
    def __init__(self):
        self._data = None
    
    def save(self, data) -> None:
        self._data = data
        print("Saved to memory")
    
    def load(self):
        return self._data


# Test all implementations
sample_data = [
    {"name": "Alice", "age": 30},
    {"name": "Bob", "age": 25}
]

print("=== JSON Storage ===")
json_store = JSONStorage("data_u8/people.json")
json_store.save(sample_data)
print(f"Loaded: {json_store.load()}\n")

print("=== CSV Storage ===")
csv_store = CSVStorage("data_u8/people.csv", ["name", "age"])
csv_store.save(sample_data)
print(f"Loaded: {csv_store.load()}\n")

print("=== Memory Storage ===")
mem_store = MemoryStorage()
mem_store.save(sample_data)
print(f"Loaded: {mem_store.load()}")

## 6.4 Programming to Interfaces

The real power of ABCs: code can depend on the **interface**, not specific implementations.

This enables:
- **Testability** — swap real storage with `MemoryStorage` for testing
- **Flexibility** — switch from JSON to a database without changing business logic
- **Clear contracts** — new developers know exactly what to implement

In [ ]:
# Function that works with ANY Storage implementation
def backup_and_verify(storage: Storage, data) -> bool:
    """Save data and verify it was stored correctly."""
    storage.save(data)
    loaded = storage.load()
    
    # Simple verification
    if len(loaded) != len(data):
        return False
    return True

# Works with any storage type!
print("JSON:", backup_and_verify(JSONStorage("data_u8/backup.json"), sample_data))
print("CSV:", backup_and_verify(CSVStorage("data_u8/backup.csv", ["name", "age"]), sample_data))
print("Memory:", backup_and_verify(MemoryStorage(), sample_data))

## 6.5 Abstract Properties

You can define abstract properties that subclasses must implement:

In [ ]:
# Abstract properties example
class Shape(ABC):
    """Abstract shape with required area and perimeter."""
    
    @property
    @abstractmethod
    def area(self) -> float:
        """Calculate area of the shape."""
        pass
    
    @property
    @abstractmethod
    def perimeter(self) -> float:
        """Calculate perimeter of the shape."""
        pass
    
    def describe(self) -> str:
        return f"Area: {self.area:.2f}, Perimeter: {self.perimeter:.2f}"


class Rectangle(Shape):
    def __init__(self, width: float, height: float):
        self.width = width
        self.height = height
    
    @property
    def area(self) -> float:
        return self.width * self.height
    
    @property
    def perimeter(self) -> float:
        return 2 * (self.width + self.height)


class Circle(Shape):
    def __init__(self, radius: float):
        self.radius = radius
    
    @property
    def area(self) -> float:
        import math
        return math.pi * self.radius ** 2
    
    @property
    def perimeter(self) -> float:
        import math
        return 2 * math.pi * self.radius


# Usage
shapes: list[Shape] = [Rectangle(4, 5), Circle(3)]
for shape in shapes:
    print(f"{shape.__class__.__name__}: {shape.describe()}")

---

# 7. Class Methods and Static Methods

## 7.1 Three Types of Methods

| Method Type | Decorator | First Parameter | Access |
|-------------|-----------|-----------------|--------|
| Instance method | (none) | `self` | Instance + class data |
| Class method | `@classmethod` | `cls` | Class data only |
| Static method | `@staticmethod` | (none) | No implicit access |

## 7.2 When to Use Each

- **Instance methods**: Most common. Require access to object state.
- **Class methods**: Alternative constructors, factory methods, accessing class-level data.
- **Static methods**: Utility functions logically grouped with the class.

In [ ]:
# Comprehensive example showing all three method types
class Person:
    """Person class demonstrating different method types."""
    
    # Class attribute - shared by all instances
    population = 0
    
    def __init__(self, name: str, birth_year: int):
        self.name = name
        self.birth_year = int(birth_year)
        Person.population += 1
    
    # Instance method - operates on self
    def introduce(self) -> str:
        """Instance method: needs self."""
        return f"Hi, I'm {self.name}, born in {self.birth_year}"
    
    def age(self, current_year: int = 2024) -> int:
        """Instance method: calculates age."""
        return current_year - self.birth_year
    
    # Class method - alternative constructor
    @classmethod
    def from_string(cls, text: str) -> "Person":
        """Class method: creates Person from 'name,year' string."""
        name, year = text.split(",")
        return cls(name.strip(), int(year.strip()))
    
    @classmethod
    def from_dict(cls, data: dict[str, str | int]) -> "Person":
        """Class method: creates Person from dictionary."""
        return cls(str(data["name"]), int(data["birth_year"]))
    
    @classmethod
    def get_population(cls) -> int:
        """Class method: access class-level data."""
        return cls.population
    
    # Static method - utility function
    @staticmethod
    def is_valid_year(year: int) -> bool:
        """Static method: validation utility."""
        return 1900 <= int(year) <= 2024
    
    @staticmethod
    def calculate_age(birth_year: int, current_year: int = 2024) -> int:
        """Static method: pure calculation, no instance needed."""
        return current_year - birth_year


# Instance method usage
p1 = Person("Alice", 1990)
print(p1.introduce())
print(f"Age: {p1.age()}")

# Class method usage - alternative constructors
p2 = Person.from_string("Bob, 1985")
p3 = Person.from_dict({"name": "Charlie", "birth_year": 2000})
print(f"\nCreated: {p2.name}, {p3.name}")
print(f"Population: {Person.get_population()}")

# Static method usage - no instance needed
print(f"\nIs 1990 valid? {Person.is_valid_year(1990)}")
print(f"Is 1800 valid? {Person.is_valid_year(1800)}")
print(f"Age for birth year 1990: {Person.calculate_age(1990)}")

## 7.3 Class Methods and Inheritance

Class methods work correctly with inheritance—`cls` refers to the actual class being called:

In [ ]:
# Class methods work correctly with inheritance
class Animal:
    def __init__(self, name: str):
        self.name = name
    
    @classmethod
    def create_baby(cls, parent_name: str) -> "Animal":
        """Creates a baby of the same type."""
        return cls(f"Baby of {parent_name}")
    
    def speak(self) -> str:
        return "..."

class Dog(Animal):
    def speak(self) -> str:
        return "Woof!"

class Cat(Animal):
    def speak(self) -> str:
        return "Meow!"

# cls refers to the actual class, not Animal
dog_baby = Dog.create_baby("Rex")  # Creates a Dog, not Animal
cat_baby = Cat.create_baby("Whiskers")  # Creates a Cat, not Animal

print(f"{dog_baby.name} says: {dog_baby.speak()}")  # Dog's speak()
print(f"{cat_baby.name} says: {cat_baby.speak()}")  # Cat's speak()
print(f"Types: {type(dog_baby).__name__}, {type(cat_baby).__name__}")

---

# 8. Exercises

The following exercises reinforce the concepts covered in this unit.

| Exercise | Topics | Difficulty |
|----------|--------|-----------|
| 1 | Properties, Encapsulation | Basic |
| 2 | Abstract Base Classes | Basic |

## Exercise 1: Product with Validated Properties

Create a `Product` class with the following requirements:

- `name` property (read-only after creation)
- `price` property with validation (must be > 0)
- `quantity` property with validation (must be >= 0)
- `total_value` computed property (price × quantity)
- `discount(percent)` method that reduces price

Test your implementation with the demo code provided.

In [ ]:
# Exercise 1: Product with Properties - Starter Code

class Product:
    """Product with validated properties."""
    
    def __init__(self, name: str, price: float, quantity: int = 0):
        self._name = name
        # TODO: Use properties for price and quantity
        self.price = price
        self.quantity = quantity
    
    @property
    def name(self) -> str:
        """Name is read-only."""
        return self._name
    
    @property
    def price(self) -> float:
        # TODO: Return internal _price
        pass
    
    @price.setter
    def price(self, value: float) -> None:
        # TODO: Validate price > 0, then set _price
        pass
    
    @property
    def quantity(self) -> int:
        # TODO: Return internal _quantity
        pass
    
    @quantity.setter
    def quantity(self, value: int) -> None:
        # TODO: Validate quantity >= 0, then set _quantity
        pass
    
    @property
    def total_value(self) -> float:
        """Computed: price × quantity."""
        # TODO: Return price * quantity
        pass
    
    def discount(self, percent: float) -> None:
        """Apply a percentage discount to price."""
        # TODO: Reduce price by percent%
        pass
    
    def __repr__(self) -> str:
        return f"Product({self.name}, ${self.price:.2f}, qty={self.quantity})"


# === Demo (uncomment after implementation) ===
# p = Product("Laptop", 999.99, 5)
# print(p)
# print(f"Total value: ${p.total_value:.2f}")
#
# p.discount(10)  # 10% off
# print(f"After 10% discount: {p}")
#
# # Test validation
# try:
#     p.price = -100  # Should raise ValueError
# except ValueError as e:
#     print(f"Validation works: {e}")

## Exercise 2: Notifier Interface with ABC

Create a notification system using Abstract Base Classes:

- `Notifier` ABC with abstract `send(recipient, message)` method
- `ConsoleNotifier` — prints to console
- `ListNotifier` — stores messages in a list (useful for testing)
- `welcome_user(notifier, username)` function

This exercise demonstrates the Dependency Inversion Principle.

In [ ]:
# Exercise 2: Notifier Interface - Starter Code

from abc import ABC, abstractmethod

class Notifier(ABC):
    """Abstract base class for notification systems."""
    
    @abstractmethod
    def send(self, recipient: str, message: str) -> None:
        """Send a message to a recipient."""
        pass


class ConsoleNotifier(Notifier):
    """Print notifications to console."""
    
    def send(self, recipient: str, message: str) -> None:
        # TODO: Print formatted message like "[To: recipient] message"
        pass


class ListNotifier(Notifier):
    """Store notifications in a list (for testing)."""
    
    def __init__(self):
        self.messages: list[tuple[str, str]] = []
    
    def send(self, recipient: str, message: str) -> None:
        # TODO: Append (recipient, message) tuple to self.messages
        pass


def welcome_user(notifier: Notifier, username: str) -> None:
    """Send welcome message using any notifier."""
    # TODO: Use notifier.send() to send "Welcome, {username}!"
    pass


# === Demo (uncomment after implementation) ===
# # Test ConsoleNotifier
# console = ConsoleNotifier()
# welcome_user(console, "Alice")
#
# # Test ListNotifier (no output, stores messages)
# test_notifier = ListNotifier()
# welcome_user(test_notifier, "Bob")
# welcome_user(test_notifier, "Charlie")
# print(f"Stored messages: {test_notifier.messages}")

---

## 📝 Summary

This unit covered the following key concepts:

**📝 Type Hints**
- Annotate functions and classes with `str`, `int`, `float`, `bool`, and collection types
- Use modern `X | None` syntax for optional values and `A | B` for union types (Python 3.10+)
- Type hints improve readability, enable IDE support, and help catch bugs early

**🔒 Encapsulation**
- Use `_attr` (protected) and `__attr` (name-mangled) conventions to protect internal state
- Python trusts developers to follow conventions rather than enforcing strict access control

**⚙️ Properties**
- Provide controlled, Pythonic attribute access with validation
- Support read-only attributes, computed values, and caching

**📦 Dataclasses**
- Use `@dataclass` to auto-generate `__init__`, `__repr__`, `__eq__` and more
- Use `field(default_factory=...)` for mutable defaults and `frozen=True` for immutability
- Combine with `__post_init__` for validation after initialization

**📋 Abstract Base Classes**
- Define contracts that implementations must follow
- Enable programming to interfaces for flexibility and testability

**🔧 Method Types**
- Use `@classmethod` for alternative constructors and class-level operations
- Use `@staticmethod` for utility functions that don't need instance or class access

---

**Next:** Unit 8b — SOLID Principles